# Read Me

For the easiest experience of running this .ipynb, please use Google Colab. All that needs to be done is to upload the included .txt files that are saved inside the "data" file within this .zip. Once the .ipynb has been uploaded, simply go to "Runtime" and select "Run all", and the model will be ready in approxametly 1 hour.


In [ ]:
import os, sys
import tensorflow as tf
import numpy as pn
import sklearn
import matplotlib as mat
import torch
import math

from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical, plot_model
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from numpy import array, asarray, zeros

In [ ]:
print(tf.__version__)
print(np.__version__)
print(mat.__version__)
print(sklearn.__version__)
print(torch.__version__)

2.8.0
1.21.5
3.2.2
1.0.2
1.10.0+cu111


In [ ]:
BATCH_SIZE = 64
EPOCHS = 20
LSTM_NODES =256
NUM_POLYS = 20000
MAX_POLY_LENGTH = 50
MAX_NUM_CHARS = 20000
EMBEDDING_SIZE = 100

In [ ]:
''' This showed me all the characters within the dataset
with open('dataset.txt', 'r') as dataset:
  all_chars = set(dataset.read().replace('\n', ''))

dataset.close()

print(all_chars)
'''
nums = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9') #numbers
ops = ('(', ')', '+', '-', '*') #operators
vars = ('h', 'n', 'o', 's', 'c', 'z', 'a', 'y', 't', 'x', 'j', 'k') #variables

In [ ]:
with open('dataset.txt', 'r') as dataset:
  raw = dataset.readlines()

factorized = []
expanded = []

for i, poly in enumerate(raw):
  raw[i] = raw[i].strip()
  equal = raw[i].find('=')
  factorized.append(raw[i][0:equal])
  expanded.append(raw[i][equal+1:])

dataset.close()

In [ ]:
def clean_data(data):
  data_clean = []
  for i, poly in enumerate(data):
    factor = ""
    j = 0
    while j < len(poly) - 1:

      if poly[j] in nums:
        if poly[j+1] not in nums:
          factor += poly[j] + ' '
        else:
          factor += poly[j]
      
      elif poly[j] in ops:
        if poly[j] is '*' and poly[j+1] is '*':
          factor += poly[j] + '* '
          j += 1
        else:
          factor += poly[j] + ' '
      
      elif poly[j] in vars:
        if poly[j] is 's' and poly[j+1] is 'i':
          factor += 'sin '
          j += 2
        elif poly[j] is 'c' and poly[j+1] is 'o':
          factor += 'cos '
          j += 2
        elif poly[j] is 't' and poly[j+1] is 'a':
          factor += 'tan '
          j += 2
        else:
          factor += 'x '
      j += 1
    if poly[-1] in vars:
      factor += 'x'
    else:
      factor += poly[-1]
    data_clean.append(factor)
  return data_clean

factors_clean = clean_data(factorized)
expandeds_clean = clean_data(expanded)

In [ ]:
equal_pairs = []
for i in range(len(factors_clean)):
  equal_pairs.append([factors_clean[i], expandeds_clean[i]])

#training: 80%, testing: 10%, validation: 10%
pairs, test_pairs = train_test_split(equal_pairs, test_size = 0.2)
validation_pairs, test_pairs = train_test_split(test_pairs, test_size = 0.5)

In [ ]:
input_polys = []
output_polys = []
output_polys_inputs = []

count = 0
for pair in pairs:
  count += 1

  if count > NUM_POLYS:
    break

  input_polys.append(pair[0])
  output_polys.append(pair[1] + ' <eos>')
  output_polys_inputs.append('<sos> ' + pair[1])

In [ ]:
input_tokenizer = Tokenizer(num_words=MAX_NUM_CHARS)
input_tokenizer.fit_on_texts(input_polys)
input_integer_seq = input_tokenizer.texts_to_sequences(input_polys)

word2idx_inputs = input_tokenizer.word_index

max_input_len = max(len(poly) for poly in input_integer_seq)

In [ ]:
output_tokenizer = Tokenizer(num_words=MAX_NUM_CHARS, filters='')
output_tokenizer.fit_on_texts(output_polys + output_polys_inputs)
output_integer_seq = output_tokenizer.texts_to_sequences(output_polys)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_polys_inputs)

word2idx_outputs = output_tokenizer.word_index

num_chars_output = len(word2idx_outputs) + 1
max_out_len = max(len(poly) for poly in output_input_integer_seq)

In [ ]:
encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)

In [ ]:
decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')

In [ ]:
embeddings_dictionary = dict()

glove = open('glove.6B.100d.txt', encoding='utf8')

for line in glove:
  records = line.split()
  word = records[0]
  vector_dimensions = asarray(records[1:], dtype='float32')
  embeddings_dictionary[word] = vector_dimensions
glove.close()

In [ ]:
num_words = min(MAX_NUM_CHARS, len(word2idx_inputs) + 1)
embedding_matrix = zeros((num_words, EMBEDDING_SIZE))

for char, index in word2idx_inputs.items():
  embedding_vector = embeddings_dictionary.get(char)
  if embedding_vector is not None:
    embedding_matrix[index] = embedding_vector

In [ ]:
embedding_layer = Embedding(num_words, EMBEDDING_SIZE, weights=[embedding_matrix], input_length=max_input_len)

In [ ]:
decoder_targets_one_hot = np.zeros((len(input_polys), max_out_len, num_chars_output), dtype='float32')

In [ ]:
decoder_output_sequences = pad_sequences(output_integer_seq, maxlen=max_out_len, padding='post')

for i, d in enumerate(decoder_output_sequences):
  for t, char in enumerate(d):
    decoder_targets_one_hot[i, t, char] = 1

In [ ]:
encoder_inputs_placeholder = Input(shape=(max_input_len,))
x = embedding_layer(encoder_inputs_placeholder)
encoder = LSTM(LSTM_NODES, return_state=True)

encoder_outputs, h, c = encoder(x)
encoder_states = [h, c]

In [ ]:
decoder_inputs_placeholder = Input(shape=(max_out_len,))

decoder_embedding = Embedding(num_chars_output, LSTM_NODES)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)

decoder_lstm = LSTM(LSTM_NODES, return_sequences=True, return_state=True)
decoder_outputs, _, _ =decoder_lstm(decoder_inputs_x, initial_state=encoder_states)

In [ ]:
decoder_dense = Dense(num_chars_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = Model([encoder_inputs_placeholder, decoder_inputs_placeholder], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
r = model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_targets_one_hot,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.1,
    validation_data=(validation_pairs)
)

Epoch 1/20
282/282 [==============================] - 18s 35ms/step - loss: 1.0896 - accuracy: 0.7666 - val_loss: 0.7459 - val_accuracy: 0.8222
Epoch 2/20
282/282 [==============================] - 10s 34ms/step - loss: 0.6761 - accuracy: 0.8297 - val_loss: 0.6491 - val_accuracy: 0.8322
Epoch 3/20
282/282 [==============================] - 8s 30ms/step - loss: 0.5961 - accuracy: 0.8438 - val_loss: 0.5748 - val_accuracy: 0.8454
Epoch 4/20
282/282 [==============================] - 8s 30ms/step - loss: 0.5269 - accuracy: 0.8590 - val_loss: 0.5281 - val_accuracy: 0.8580
Epoch 5/20
282/282 [==============================] - 8s 29ms/step - loss: 0.4758 - accuracy: 0.8697 - val_loss: 0.4805 - val_accuracy: 0.8650
Epoch 6/20
282/282 [==============================] - 8s 29ms/step - loss: 0.4316 - accuracy: 0.8787 - val_loss: 0.4229 - val_accuracy: 0.8797
Epoch 7/20
282/282 [==============================] - 8s 29ms/step - loss: 0.3884 - accuracy: 0.8880 - val_loss: 0.3930 - val_accuracy: 0.88

In [ ]:
encoder_model = Model(encoder_inputs_placeholder, encoder_states)

In [ ]:
decoder_state_input_h = Input(shape=(LSTM_NODES,))
decoder_state_input_c = Input(shape=(LSTM_NODES,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [ ]:
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

In [ ]:
decoder_outputs, h, c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)

In [ ]:
decoder_states = [h, c]
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

In [ ]:
idx2word_input = {v:k for k, v in word2idx_inputs.items()}
idx2word_target = {v:k for k, v in word2idx_outputs.items()}

In [ ]:
def expand_poly(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<sos>']
    eos = word2idx_outputs['<eos>']
    output_sentence = []

    for _ in range(max_out_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        idx = np.argmax(output_tokens[0, 0, :])

        if eos == idx:
            break

        word = ''

        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)

        target_seq[0, 0] = idx
        states_value = [h, c]

    return ' '.join(output_sentence)

In [ ]:
i = np.random.choice(len(input_polys))
input_seq = encoder_input_sequences[i:i+1]
translation = expand_poly(input_seq)
print('-')
print('Input:', input_polys[i])
print('Response:', translation)

-
Input: ( x + 6 ) * ( 7 * x + 24 )
Response: 7 * x ** 2 + 54 * x - 144


In [ ]:
test_x = []
test_y = []

for i, pair in enumerate(test_pairs):
  test_x.append(pair[0])
  test_y.append(pair[1])

In [ ]:
test_tokenizer = Tokenizer(num_words=MAX_NUM_CHARS)
test_tokenizer.fit_on_texts(test_x)
test_integer_seq = input_tokenizer.texts_to_sequences(test_x)

word2idx_test = test_tokenizer.word_index

max_test_len = max(len(poly) for poly in test_integer_seq)

#Report

When I initially saw the problem, one of the first thoughts of how to solve it would be something similar to a translation problem. I saw no difference between translating from english to french and translating from factorized to expanded form. With some research, I was able to determine that the NLP model that would be most applicable to this specific problem was the Seq2Seq model, as this is the most commonly used model in NLP problems. Seq2Seq is a type of Recurrent Neural Network, specifically a Long Short Term Memory network. In a previous project, I had created a Bidirection LSTM using the Keras package from tensorflow, so with some additional research I found how to implement a Seq2Seq model using Keras.

The biggest problem I had was finding a way to prepare the data so that I could feed the training data into a model made for human language and allow it to predict its outcome. So to solve this, I decided to try and structure the data like a sentence. I treated variables, numbers, and logical operators like words in a sentence, and placed spaces between each.

Knowing that all the polynomials were single variable polynomials, I decided to replace all variables within the data with "x" in order to keep some consistency, which would assist greatly in training.